In [9]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, fftshift, ifft2, ifftshift
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Load original image ---
original_img = Image.open("wave.jpeg").convert('RGB')
original_img_array = np.asarray(original_img).astype(float)

# --- Function to tile image n x n times ---
def tile_image(img_array, n):
    return np.tile(img_array, (n, n, 1))

# --- Reconstruct image from full FFT (no compression) ---
def reconstruct_full_fft(channel):
    fft_channel = fft2(channel)
    fft_shifted = fftshift(fft_channel)
    reconstructed = np.real(ifft2(ifftshift(fft_shifted)))
    return reconstructed, fft_shifted  # Return the shifted FFT for magnitude display

# --- Update function without compression ---
def update_tile_only(tiles):
    tiled_img_array = tile_image(original_img_array, tiles)
    h, w = tiled_img_array.shape[:2]

    rec_r, fft_shifted_r = reconstruct_full_fft(tiled_img_array[:, :, 0])
    rec_g, fft_shifted_g = reconstruct_full_fft(tiled_img_array[:, :, 1])
    rec_b, fft_shifted_b = reconstruct_full_fft(tiled_img_array[:, :, 2])

    reconstructed_image = np.stack((rec_r, rec_g, rec_b), axis=-1)

    # --- Calculate the magnitude of the FFT and shift it to center ---
    mag_r = np.abs(fft_shifted_r) + 1
    mag_g = np.abs(fft_shifted_g) + 1
    mag_b = np.abs(fft_shifted_b) + 1

    total_mag = mag_r + mag_g + mag_b

    # Display
    plt.figure(figsize=(18, 8))

    plt.subplot(1, 3, 1)
    plt.imshow(tiled_img_array.astype(np.uint8))
    plt.title(f"Tiled Image ({tiles}×{tiles})")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(np.log(total_mag), cmap='gray', extent=[-w//2, w//2, -h//2, h//2])
    plt.title("Full FFT Magnitude (No Compression)")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(np.clip(reconstructed_image, 0, 255).astype(np.uint8))
    plt.title("Reconstructed Image (Full FFT)")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# --- Widget ---
tile_slider_only = widgets.IntSlider(
    value=1, min=1, max=5,
    description='Tile n×n:', continuous_update=False
)

out_tile_only = widgets.interactive_output(update_tile_only, {
    'tiles': tile_slider_only
})

display(widgets.VBox([tile_slider_only]), out_tile_only)


Output()

In [11]:
# --- Function to compress a single channel using top % of FFT magnitude ---
def compress_channel(channel, percentage):
    h, w = channel.shape
    fft_channel = fft2(channel)
    fft_shifted = fftshift(fft_channel)

    # Flatten and get threshold
    flat_fft = fft_shifted.flatten()
    magnitudes = np.abs(flat_fft)
    threshold_index = int((1 - percentage / 100.0) * len(magnitudes))
    threshold_index = max(threshold_index, 1)
    threshold = np.partition(magnitudes, threshold_index)[threshold_index]

    # Apply mask
    mask = magnitudes >= threshold
    compressed_fft = np.zeros_like(flat_fft, dtype=complex)
    compressed_fft[mask] = flat_fft[mask]
    compressed_fft = compressed_fft.reshape(h, w)

    # Inverse FFT
    reconstructed = np.real(ifft2(ifftshift(compressed_fft)))

    return reconstructed, (np.abs(fft_shifted) + 1), np.log(np.abs(compressed_fft) + 1), np.count_nonzero(mask)

# --- Compression label ---
compression_label = widgets.HTML()

# --- Update function with compression ---
def update_compression(percent):

    h, w = original_img_array.shape[:2]

    rec_r, full_mag_r, filt_mag_r, used_r = compress_channel(original_img_array[:, :, 0], percent)
    rec_g, full_mag_g, filt_mag_g, used_g = compress_channel(original_img_array[:, :, 1], percent)
    rec_b, full_mag_b, filt_mag_b, used_b = compress_channel(original_img_array[:, :, 2], percent)

    reconstructed_image = np.stack((rec_r, rec_g, rec_b), axis=-1)
    filt_mag = filt_mag_r + filt_mag_g + filt_mag_b

    total_coeffs = 3 * h * w
    used_coeffs = used_r + used_g + used_b
    estimated_size_kb = (8 * used_coeffs) / 1024
    estimated_uncompressed_size_kb = (8 * total_coeffs) / 1024

    compression_label.value = (
         f"<b>Uncompressed Size:</b> {estimated_uncompressed_size_kb:.2f} KB<br>"
        f"<b>Compressed Size:</b> {estimated_size_kb:.2f} KB"
    )

    # Display
    plt.figure(figsize=(18, 8))

    plt.subplot(1, 3, 1)
    plt.imshow(original_img_array.astype(np.uint8))
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(filt_mag, cmap='gray', extent=[-w//2, w//2, -h//2, h//2])
    plt.title(f"Compressed FFT Magnitude\nTop {percent:.2f}% Kept")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(np.clip(reconstructed_image, 0, 255).astype(np.uint8))
    plt.title("Reconstructed Image (Compressed)")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# --- Widgets ---
percent_slider = widgets.FloatSlider(
    value=5.0, min=0.01, max=10.0, step=0.01,
    description='Top % Coeff:', continuous_update=False
)

out_compression = widgets.interactive_output(update_compression, {
    'percent': percent_slider
})

display(widgets.VBox([percent_slider, compression_label]), out_compression)


Output()

In [13]:
def add_noise(image, noise_level):
    noise = np.random.normal(0, noise_level, image.shape)
    noisy_image = np.clip(image + noise, 0, 255)
    return noisy_image.astype(np.uint8)


def update_with_noise_and_compression(noise_level, compression_percent):

    # Add noise to the original image
    noisy_img_array = add_noise(original_img_array, noise_level)
    h, w = noisy_img_array.shape[:2]

    # Compress the noisy image channels (R, G, B)
    rec_r, full_mag_r, filt_mag_r, used_r = compress_channel(noisy_img_array[:, :, 0], compression_percent)
    rec_g, full_mag_g, filt_mag_g, used_g = compress_channel(noisy_img_array[:, :, 1], compression_percent)
    rec_b, full_mag_b, filt_mag_b, used_b = compress_channel(noisy_img_array[:, :, 2], compression_percent)

    reconstructed_image = np.stack((rec_r, rec_g, rec_b), axis=-1)
    filt_mag = filt_mag_r + filt_mag_g + filt_mag_b

    total_coeffs = 3 * h * w
    used_coeffs = used_r + used_g + used_b
    estimated_size_kb = (8 * used_coeffs) / 1024
    estimated_uncompressed_size_kb = (8 * total_coeffs) / 1024

    compression_label.value = (
         f"<b>Uncompressed Size:</b> {estimated_uncompressed_size_kb:.2f} KB<br>"
        f"<b>Compressed Size:</b> {estimated_size_kb:.2f} KB"
    )

    # Recalculate the magnitude of the FFT based on the current noisy image
    mag_r = np.abs(full_mag_r) + 1
    mag_g = np.abs(full_mag_g) + 1
    mag_b = np.abs(full_mag_b) + 1

    total_mag = mag_r + mag_g + mag_b

    # Display
    plt.figure(figsize=(18, 8))

    # Noisy image
    plt.subplot(1, 3, 1)
    plt.imshow(noisy_img_array.astype(np.uint8))
    plt.title(f"Noisy Image (Noise Level: {noise_level})")
    plt.axis('off')

    # FFT Magnitude of compressed image (k-space)
    plt.subplot(1, 3, 2)
    plt.imshow(filt_mag, cmap='gray', extent=[-w//2, w//2, -h//2, h//2])
    plt.title(f"Compressed FFT Magnitude\nTop {compression_percent:.2f}% Kept")
    plt.axis('off')

    # Reconstructed Image
    plt.subplot(1, 3, 3)
    plt.imshow(np.clip(reconstructed_image, 0, 255).astype(np.uint8))
    plt.title("Reconstructed Image (With Noise & Compression)")
    plt.axis('off')

    plt.tight_layout()
    plt.show()


# Update sliders with more precision
noise_slider = widgets.FloatSlider(
    value=10.0, min=0.0, max=5000.0, step=0.01,  # More precision in steps
    description='Noise Level:', continuous_update=False,
    readout_format='.2f'  # To display values with 2 decimal places
)

percent_slider = widgets.FloatSlider(
    value=5.0, min=0.001, max=100.0, step=0.0001,  # Increased precision in steps
    description='Top % Coeff:', continuous_update=False,
    readout_format='.4f'  # Display up to 4 decimal places
)

# Combine both sliders and create output for interactivity
out_with_noise_and_compression = widgets.interactive_output(update_with_noise_and_compression, {
    'noise_level': noise_slider,
    'compression_percent': percent_slider
})

compression_label = widgets.HTML(value="")

# Display the updated widgets
display(widgets.VBox([noise_slider, percent_slider, compression_label]), out_with_noise_and_compression)


Output()

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft2, fftshift, ifft2, ifftshift
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Functions ---
def create_bar_mask(h, w, x_thickness, y_thickness, x_center, y_center):
    mask = np.zeros((h, w), dtype=float)

    # Horizontal bar (thick in y-direction, spans full width)
    y_half = int(y_thickness / 2)
    y_start = max(int(y_center - y_half), 0)
    y_end = min(int(y_center + y_half + 1), h)
    mask[y_start:y_end, :] = 1

    # Vertical bar (thick in x-direction, spans full height)
    x_half = int(x_thickness / 2)
    x_start = max(int(x_center - x_half), 0)
    x_end = min(int(x_center + x_half + 1), w)
    mask[:, x_start:x_end] = 1

    return mask

def process_channel(channel, mask=None):
    fft_channel = fft2(channel)
    fft_shifted = fftshift(fft_channel)
    if mask is not None:
        fft_shifted *= mask
    magnitude = np.abs(fft_shifted)
    phase = np.angle(fft_shifted)
    return fft_shifted, magnitude, phase

# --- Use the Image from the First Cell ---
h, w = original_img_array.shape[:2]

# --- Interactive Update Function ---
def update(x_thickness, y_thickness, x_center, y_center):

    mask = create_bar_mask(h, w, x_thickness, y_thickness, x_center, y_center)

    fft_r, mag_r, _ = process_channel(original_img_array[:, :, 0], mask)
    fft_g, mag_g, _ = process_channel(original_img_array[:, :, 1], mask)
    fft_b, mag_b, _ = process_channel(original_img_array[:, :, 2], mask)

    combined_magnitude = mag_r + mag_g + mag_b

    reconstructed_r = np.real(ifft2(ifftshift(fft_r)))
    reconstructed_g = np.real(ifft2(ifftshift(fft_g)))
    reconstructed_b = np.real(ifft2(ifftshift(fft_b)))

    reconstructed_image = np.stack((reconstructed_r, reconstructed_g, reconstructed_b), axis=-1)

    # Plot
    plt.figure(figsize=(18, 8))

    plt.subplot(1, 3, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(np.log(combined_magnitude + 1), cmap='gray')
    plt.title(f"Magnitude Spectrum\nX Thick: {x_thickness}, Y Thick: {y_thickness}\nX Ctr: {x_center}, Y Ctr: {y_center}")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(np.clip(reconstructed_image, 0, 255).astype(np.uint8))
    plt.title("Reconstructed Image")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# --- Create Sliders ---
x_thickness_slider = widgets.FloatSlider(
    value=10.0,
    min=0.0,
    max=w,
    step=1.0,
    description='X Thickness:',
    continuous_update=False
)

y_thickness_slider = widgets.FloatSlider(
    value=10.0,
    min=0.0,
    max=h,
    step=1.0,
    description='Y Thickness:',
    continuous_update=False
)

x_center_slider = widgets.FloatSlider(
    value=w // 2,
    min=0,
    max=w - 1,
    step=1.0,
    description='X Center:',
    continuous_update=False
)

y_center_slider = widgets.FloatSlider(
    value=h // 2,
    min=0,
    max=h - 1,
    step=1.0,
    description='Y Center:',
    continuous_update=False
)

# --- Display Interactive UI ---
ui = widgets.VBox([
    x_thickness_slider,
    y_thickness_slider,
    x_center_slider,
    y_center_slider
])

out = widgets.interactive_output(update, {
    'x_thickness': x_thickness_slider,
    'y_thickness': y_thickness_slider,
    'x_center': x_center_slider,
    'y_center': y_center_slider
})

display(ui, out)


Output()

In [19]:
# --- Functions for Radial Mask ---
def create_radial_mask(h, w, radius, x_center, y_center, invert=False):
    y, x = np.ogrid[:h, :w]
    dist = np.sqrt((x - x_center)**2 + (y - y_center)**2)
    mask = dist <= radius  # Inside the circle

    if invert:
        mask = np.logical_not(mask)  # Invert the mask

    return mask.astype(float)

# --- Update Function for Radial Filter ---
def update_radial(radius, x_center, y_center, invert_mask):

    # Create radial mask
    mask = create_radial_mask(h, w, radius, x_center, y_center, invert_mask)

    # Process the channels
    fft_r, mag_r, _ = process_channel(original_img_array[:, :, 0], mask)
    fft_g, mag_g, _ = process_channel(original_img_array[:, :, 1], mask)
    fft_b, mag_b, _ = process_channel(original_img_array[:, :, 2], mask)

    combined_magnitude = mag_r + mag_g + mag_b

    # Reconstruct the image
    reconstructed_r = np.real(ifft2(ifftshift(fft_r)))
    reconstructed_g = np.real(ifft2(ifftshift(fft_g)))
    reconstructed_b = np.real(ifft2(ifftshift(fft_b)))

    reconstructed_image = np.stack((reconstructed_r, reconstructed_g, reconstructed_b), axis=-1)

    # Plot
    plt.figure(figsize=(18, 8))

    plt.subplot(1, 3, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(np.log(combined_magnitude + 1), cmap='gray')
    plt.title(f"Magnitude Spectrum\nRadius: {radius}\nX Ctr: {x_center}, Y Ctr: {y_center}\nInvert: {invert_mask}")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(np.clip(reconstructed_image, 0, 255).astype(np.uint8))
    plt.title("Reconstructed Image")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# --- Create Sliders for Radial Filter ---
radius_slider = widgets.FloatSlider(
    value=50.0,
    min=0.0,
    max=min(h, w) // 2,
    step=1.0,
    description='Radius:',
    continuous_update=False
)

x_center_slider_radial = widgets.FloatSlider(
    value=w // 2,
    min=0,
    max=w - 1,
    step=1.0,
    description='X Center:',
    continuous_update=False
)

y_center_slider_radial = widgets.FloatSlider(
    value=h // 2,
    min=0,
    max=h - 1,
    step=1.0,
    description='Y Center:',
    continuous_update=False
)

invert_toggle = widgets.Checkbox(
    value=False,
    description='Invert Mask',
    continuous_update=False
)

# --- Display Interactive UI for Radial Filter ---
ui_radial = widgets.VBox([
    radius_slider,
    x_center_slider_radial,
    y_center_slider_radial,
    invert_toggle
])

out_radial = widgets.interactive_output(update_radial, {
    'radius': radius_slider,
    'x_center': x_center_slider_radial,
    'y_center': y_center_slider_radial,
    'invert_mask': invert_toggle
})

display(ui_radial, out_radial)


Output()

In [102]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, clear_output

# --- Function to create quadrant masks ---
def create_quadrant_mask(h, w, quadrant, invert=False):
    y, x = np.ogrid[:h, :w]

    # Define the quadrants
    if quadrant == 1:
        mask = (x < w // 2) & (y < h // 2)  # Top-left
    elif quadrant == 2:
        mask = (x >= w // 2) & (y < h // 2)  # Top-right
    elif quadrant == 3:
        mask = (x < w // 2) & (y >= h // 2)  # Bottom-left
    elif quadrant == 4:
        mask = (x >= w // 2) & (y >= h // 2)  # Bottom-right

    if invert:
        mask = np.logical_not(mask)  # Invert the mask

    return mask.astype(float)

# --- Update function for Quadrant Mask ---
def update_quadrant_mask(q1, q2, q3, q4, invert_mask):

    # Combine the masks for the selected quadrants
    mask = np.zeros((h, w), dtype=float)

    if q1:
        mask += create_quadrant_mask(h, w, 1, invert=invert_mask)
    if q2:
        mask += create_quadrant_mask(h, w, 2, invert=invert_mask)
    if q3:
        mask += create_quadrant_mask(h, w, 3, invert=invert_mask)
    if q4:
        mask += create_quadrant_mask(h, w, 4, invert=invert_mask)

    # Process the channels
    fft_r, mag_r, _ = process_channel(original_img_array[:, :, 0], mask)
    fft_g, mag_g, _ = process_channel(original_img_array[:, :, 1], mask)
    fft_b, mag_b, _ = process_channel(original_img_array[:, :, 2], mask)

    combined_magnitude = mag_r + mag_g + mag_b

    # Reconstruct the image
    reconstructed_r = np.real(ifft2(ifftshift(fft_r)))
    reconstructed_g = np.real(ifft2(ifftshift(fft_g)))
    reconstructed_b = np.real(ifft2(ifftshift(fft_b)))

    reconstructed_image = np.stack((reconstructed_r, reconstructed_g, reconstructed_b), axis=-1)

    # Plot
    plt.figure(figsize=(18, 8))

    plt.subplot(1, 3, 1)
    plt.imshow(original_img)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(np.log(combined_magnitude + 1), cmap='gray')
    plt.title(f"Magnitude Spectrum\nInvert: {invert_mask}")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(np.clip(reconstructed_image, 0, 255).astype(np.uint8))
    plt.title("Reconstructed Image")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# --- Create checkboxes for each quadrant ---
q1_checkbox = widgets.Checkbox(value=False, description="Quadrant 1 (Top-left)", continuous_update=False)
q2_checkbox = widgets.Checkbox(value=False, description="Quadrant 2 (Top-right)", continuous_update=False)
q3_checkbox = widgets.Checkbox(value=False, description="Quadrant 3 (Bottom-left)", continuous_update=False)
q4_checkbox = widgets.Checkbox(value=False, description="Quadrant 4 (Bottom-right)", continuous_update=False)

invert_toggle = widgets.Checkbox(value=False, description="Invert Mask", continuous_update=False)

# --- Display Interactive UI for Quadrant Mask ---
ui_quadrants = widgets.VBox([q1_checkbox, q2_checkbox, q3_checkbox, q4_checkbox, invert_toggle])

out_quadrants = widgets.interactive_output(update_quadrant_mask, {
    'q1': q1_checkbox,
    'q2': q2_checkbox,
    'q3': q3_checkbox,
    'q4': q4_checkbox,
    'invert_mask': invert_toggle
})

display(ui_quadrants, out_quadrants)


Output()